# Wikidata Download - Reconcile Works

This script adds the Wikidata Q number from the Wikdiata API to a TSV file for the Work title if the author has already been reconciled. 

This script modifies the TSV file itself in batches, should the script timeout or other error you can rerun it and it will pickup where it left off, always run it on a backup of your orginal data files.

It creates a new column in the file `wikidata_work_qid` which holds the wikidata QId

In [ ]:
import pandas as pd
import requests
import time
import string
import unicodedata

id_column_author_qid## Config
Set these variables below based on your setup

`path_to_tsv` - the path to the TSV file you want to run it on

`id_column_author_qid` - the name of the column header that contains author wikidata qid

`id_column_work_title` - the name of the column header that contains the title of the work

`user_agent` - this is the value put into the headers on each request, it is good practice to identifiy your client/project when working with open free APIs

`pause_between_req` - number of seconds to wait between each API call



In [ ]:
path_to_tsv = "/Users/m/Downloads/data-tmp/hathitrust_post45fiction_metadata.tsv"

id_column_author_qid = "wikidata_qid"
id_column_work_title = "shorttitle"
user_agent = 'USER thisismattmiller - Test Script'
pause_between_req = 0


In [ ]:
def add_qid(d):



    # if there is already a value skip it
    if 'wikidata_work_qid' in d:
        if type(d['wikidata_work_qid']) == str:        
            # print('Skip',d[id_column_name])
            return d

    if pd.isnull(d[id_column_author_qid]) == False and pd.isnull(d[id_column_work_title]) == False:
        
        sparql = f"""
            SELECT ?item ?itemLabel 
            WHERE 
            {{

              ?item wdt:P50 wd:{d[id_column_author_qid]}.
              SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
            }}
        """
        params = {
            'query' : sparql
        }

        headers = {
            'Accept' : 'application/json',
            'User-Agent': user_agent
        }
        url = "https://query.wikidata.org/sparql"

        r = requests.get(url, params=params, headers=headers)

        data = r.json()



        # did we get any results
        works = []
        work_titles = {}
        if len(data['results']['bindings']) > 0:
          for result in data['results']['bindings']:

            qid = result['item']['value'].split('/')[-1]
            itemLabel = result['itemLabel']['value']
            if itemLabel not in work_titles:
                works.append({'qid':qid,'title':itemLabel})
                work_titles[itemLabel] = qid
            

        for w in works:

            if normalize_string(w['title']) == normalize_string(d[id_column_work_title]):
                print("exact match", w['title'], ' === ', d[id_column_work_title], w['qid'])
                d['wikidata_work_qid'] = w['qid']
                return d
        

        starts_with_match = []
        for w in works:
            if normalize_string(w['title']).startswith(normalize_string(d[id_column_work_title])):
                starts_with_match.append(w)

            if normalize_string(d[id_column_work_title]).startswith(normalize_string(w['title'])):
                starts_with_match.append(w)
        

        #remove Q108005565
        
        if len(starts_with_match) == 1:
            if len(starts_with_match[0]['title']) > 3 and len(d[id_column_work_title]) > 3:                    
                print("starts with match", starts_with_match[0]['title'], ' === ', d[id_column_work_title], w['qid'])
                d['wikidata_work_qid'] = starts_with_match[0]['qid']
                return d

       
    time.sleep(pause_between_req)

    return d

def normalize_string(s):
    s = str(s)
    s = s.translate(str.maketrans('', '', string.punctuation))
    s = " ".join(s.split())
    s = s.lower()
    s = s.casefold()
    s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
    s = s.replace('the','')
    return s.strip()

def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]


In [ ]:
# load the tsv
df = pd.read_csv(path_to_tsv, sep='\t', header=0, low_memory=False)


# we are going to split the dataframe into chunks so we can save our progress as we go but don't want to save the entire file on on every record operation
n = 500  #chunk row size
list_df = [df[i:i+n] for i in range(0,df.shape[0],n)]

# loop through each chunk
for idx, df_chunk in enumerate(list_df):

    # if idx < 25:
    #     continue

    print("Working on chunk ", idx, 'of', len(list_df))
    list_df[idx] = list_df[idx].apply(lambda d: add_qid(d),axis=1 )  
    

    reformed_df = pd.concat(list_df)
    reformed_df.to_csv(path_to_tsv, sep='\t')


